# Part 02: Export nodes and select working streets
michael babb  
2024 11 24

In [48]:
# standard
import os

In [49]:
# external
import geopandas as gpd
import numpy as np
import pandas as pd
import shapely
from shapely.geometry import LineString, Point

# load the streetnetwork

In [50]:
# file path
file_path = 'H:/project/seattle_streets/data/' 

In [51]:
file_name = 'Street_Network_Database_Complete.gpkg'

In [52]:
fpn = os.path.join(file_path, file_name)

In [53]:
gdf = gpd.read_file(filename = fpn)

# generate street end vertices - these will be helpful with identification in subsequent steps

In [54]:
def generate_street_end_vertices(gdf:gpd.GeoDataFrame) -> gpd.GeoDataFrame | gpd.GeoDataFrame:
    """ Generate a geodataframe with the street ends.
    Args:
        gdf (gpd.GeoDataFrame): Streets

    Returns:
        gpd.GeoDataFrame | gpd.GeoDataFrame: street network, street ends
    """

    
    
    # export the vertex of each street end. 
    gdf['start_coord'] = gdf['geometry'].map(lambda x: x.coords[0])
    
    gdf['end_coord'] = gdf['geometry'].map(lambda x: x.coords[-1])
    
    f_node_gdf = gdf[['f_intr_id', 'start_coord']].copy().rename(columns = {'f_intr_id':'node_id',
                                                                           'start_coord':'coord'})
    t_node_gdf = gdf[['t_intr_id', 'end_coord']].copy().rename(columns = {'t_intr_id':'node_id',
                                                                         'end_coord':'coord'})
    
    # stack the datagrame
    node_df = pd.concat(objs = [f_node_gdf, t_node_gdf]).drop_duplicates()
    
    # make a gdf
    node_gdf = gpd.GeoDataFrame(data = node_df, geometry =
                                node_df['coord'].map(lambda x: Point(x)), crs = 'epsg:4326')
    
    node_gdf = node_gdf.drop(labels = ['coord'], axis = 1)

    # remove the start and end coordinates
    gdf = gdf.drop(labels = ['start_coord', 'end_coord'], axis = 1)
    
    return gdf, node_gdf

In [55]:
# can we generate the vertices? yes... but let's check what type of geometry we are working with
gdf['geometry'].map(lambda x: x.geom_type).value_counts()

geometry
MultiLineString    34378
Name: count, dtype: int64

In [56]:
# everything is a multiline string??? is that necessary?

In [57]:
test_gdf = gdf.explode()

In [58]:
test_gdf.shape

(34378, 38)

In [59]:
# nope! if MultiLineStrings were necessary, the number of records would increase. Let's use the .explode() function to convert things to a LineString

In [60]:
gdf = test_gdf.copy()

In [61]:
gdf['geometry'].geom_type.value_counts()

LineString    34378
Name: count, dtype: int64

In [62]:
gdf, node_gdf = generate_street_end_vertices(gdf = gdf)

In [63]:
output_file_name = 'Street_Network_Nodes.gpkg'

In [64]:
ofpn = os.path.join(file_path, output_file_name)

In [65]:
node_gdf.to_file(filename = ofpn)

# import the classified streets - this classifies streets without a classification. 

In [66]:
file_name = 'blank_street_type_modified.xlsx'

In [67]:
fpn = os.path.join(file_path, file_name)

In [68]:
blank_street_type_df = pd.read_excel(io = fpn)

In [69]:
blank_street_type_df.head()

,ord_stname_concat,ord_street_type_fix
0,NE SUNRISE VISTA,ST
1,NW ESPLANADE,ST
2,BROADWAY E,ST
3,FAUNTLEE CREST SW,ST
4,NE FOREST VISTA,ST


In [70]:
gdf = pd.merge(left = gdf, right = blank_street_type_df, how = 'left')

In [71]:
gdf.loc[gdf['ord_street_type'].isna(), 'ord_street_type'] = gdf.loc[gdf['ord_street_type'].isna(), 'ord_street_type_fix']

In [72]:
gdf['ord_street_type'].unique()

array(['AVE', 'CT', 'ST', 'PL', 'LN', 'RD', 'DR', 'WAY', 'PKWY', 'BLVD',
       'CIR', 'TER', 'AL', 'RP', 'LOOP', 'SR', 'IS', 'OP', 'TRL', 'RR',
       'BR', 'WKWY', 'ET', 'STCR', 'VIEW', 'FLYOVER', 'HWY', 'RN', 'PZ',
       'MALL'], dtype=object)

In [73]:
gdf = gdf.drop(labels = ['ord_street_type_fix'], axis = 1)

# keep only streets in Seattle

In [74]:
gdf = gdf.loc[(gdf['l_city'] == 'SEATTLE') |
(gdf['r_city'] == 'SEATTLE'), :].copy()

# write the full seattle streets to disk

In [75]:
output_file_name = 'Street_Network_Database_Seattle_Full.gpkg'

In [76]:
ofpn = os.path.join(file_path, output_file_name)

In [77]:
gdf.to_file(filename = ofpn, driver = 'GPKG', index = False)

# export the individual street portions to make the different sections

In [78]:
# after performing some analysis in GIS, we can quickly identify the central streets

In [79]:
file_name = 'Street_Network_Database_Seattle_Central_Streets.csv'

In [80]:
fpn = os.path.join(file_path, file_name)

In [81]:
cs_df = pd.read_csv(filepath_or_buffer=fpn)

In [82]:
cs_df = cs_df.drop(labels = ['fid'], axis =  1)

In [83]:
# working_gdf
# winnow down the list of streets to get what we're looking for 

q_str = 'snd_feacode in (1, 5) and st_code in (0, ) and city_portion != \'WB\''
wgdf = gdf.query(expr = q_str)

# draw a convex hull around each group of city streets

In [84]:
data_list = []
geom_list = []
for cp in wgdf['city_portion'].unique().tolist():
    print(cp)
    if cp == 'CNTR':
        temp_geoms = wgdf.loc[(wgdf['snd_id'].isin(cs_df['snd_id'])) &
        (wgdf['city_portion'] == cp), 'geometry']
    else:
        temp_geoms = wgdf.loc[wgdf['city_portion'] == cp, 'geometry']
    
    # create the convex hull
    geoms = shapely.unary_union(geometries = temp_geoms)
    cvx_hull = shapely.convex_hull(geometry = geoms)
    temp_list = [cp, 'convex']
    data_list.append(temp_list)
    geom_list.append(cvx_hull)

S
E
NW
N
CNTR
NE
SW
W


In [85]:
output_gdf = gpd.GeoDataFrame(data = data_list, columns = ['city_portion', 'hull_type'], geometry = geom_list, crs = 'epsg:4326')

In [86]:
file_name = 'city_sections.gpkg'

In [87]:
ofpn = os.path.join(file_path, file_name)

In [88]:
output_gdf.to_file(filename = ofpn, driver = 'GPKG', index = False)

# remove "streets"  are really short turnouts and walking paths

AL: alley  
TRL: trail  
OP: overpass  
IS: interstate  
SR: staterout  
RR: rail  
FLYOVER: flyover  
STCR: streetcar  
ET: extension  
RN: turn  
RP: highway ramps  
WKWY: walkways  

In [89]:
# remove streets of the following type
street_type_to_remove = [ 'AL', 'TRL', 'OP', 'IS', 'SR', 'RR', 'FLYOVER', 'STCR', 'ET', 'RN', 'RP', 'WKWY']

In [90]:
# before...
gdf.shape

(27891, 38)

In [ ]:
# remove...
gdf = gdf.loc[-gdf['ord_street_type'].isin(street_type_to_remove), :].copy()

In [92]:
# after..
gdf.shape

(26881, 38)

In [93]:
gdf['ord_street_type'].unique()

array(['AVE', 'ST', 'PL', 'LN', 'RD', 'WAY', 'PKWY', 'DR', 'CT', 'BLVD',
       'CIR', 'TER', 'BR', 'VIEW', 'LOOP', 'PZ', 'MALL'], dtype=object)

# save as a gdf - overwrite

In [94]:
# file path
output_file_name = 'Street_Network_Database_Seattle_working.gpkg'

In [95]:
ofpn = os.path.join(file_path, output_file_name)

In [96]:
gdf.to_file(filename = ofpn, driver = 'GPKG', index = False)